In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np

import pickle

from epimodel.pymc3_models.cm_effect.datapreprocessor import DataPreprocessor
from os import walk

import re

from matplotlib.font_manager import FontProperties
fp2 = FontProperties(fname=r"../../fonts/Font Awesome 5 Free-Solid-900.otf")

setting environment variables properly now done.


In [3]:
dp = DataPreprocessor(drop_HS=True)
data = dp.preprocess_data("double_entry_final.csv", last_day="2020-05-30", schools_unis="lol")

INFO:epimodel.pymc3_models.cm_effect.datapreprocessor:Dropping Travel Screen/Quarantine
INFO:epimodel.pymc3_models.cm_effect.datapreprocessor:Dropping Travel Bans
INFO:epimodel.pymc3_models.cm_effect.datapreprocessor:Dropping Public Transport Limited
INFO:epimodel.pymc3_models.cm_effect.datapreprocessor:Dropping Internal Movement Limited
INFO:epimodel.pymc3_models.cm_effect.datapreprocessor:Dropping Public Information Campaigns
INFO:epimodel.pymc3_models.cm_effect.datapreprocessor:Dropping Symptomatic Testing
INFO:epimodel.pymc3_models.cm_effect.datapreprocessor:Performing Smoothing
INFO:epimodel.pymc3_models.cm_effect.datapreprocessor:Skipping smoothing Albania
INFO:epimodel.pymc3_models.cm_effect.datapreprocessor:Skipping smoothing Georgia
INFO:epimodel.pymc3_models.cm_effect.datapreprocessor:Skipping smoothing Iceland
INFO:epimodel.pymc3_models.cm_effect.datapreprocessor:Skipping smoothing Latvia
INFO:epimodel.pymc3_models.cm_effect.datapreprocessor:Skipping smoothing New Zealand
IN

In [4]:
default_res = pickle.load(open("final_full.pkl", "rb")).CMReduction

In [5]:
exp_dir = "../../server/additional_exps"

def load_exp(exp_num, local=False):
    if not local:
        return pickle.load(open(f"{exp_dir}/exp_{exp_num}.pkl", "rb"))
    else:
        return pickle.load(open(f"../../additional_exps/exp_{exp_num}.pkl", "rb"))

colors = [*sns.color_palette("colorblind"), *sns.color_palette("bright")]

In [6]:
derived_features = [ #()"Healthcare Infection Control", [0]),
 ("Mask-wearing", [0]),
 ("Gatherings <1000", [1]),
 ("Gatherings <100", [2, 1]),
 ("Gatherings <10", [3, 2, 1]),
 ("Some businesses closed", [4]),
 ("Most businesses closed", [4, 5]),
 ("Schools and universities\nclosed", [6, 7]),
 ("Stay-at-home order", [8]),
 ("Schools closed", [6]),
 ("Universities closed", [7]),
 ]


cm_plot_style = [
#             ("\uf7f2", "tab:red"), # hospital symbol
            ("\uf963", "black"), # mask
            ("\uf0c0", "darkgrey"), # ppl
            ("\uf0c0", "dimgrey"), # ppl
            ("\uf0c0", "black"), # ppl
            ("\uf07a", "tab:orange"), # shop 1
            ("\uf07a", "tab:red"), # shop2
            ("\uf549", "black"), # school
            ("\uf19d", "black"), # university
            ("\uf965", "black") # home
]

def produce_ranges(trace):
    means = np.mean(trace, axis=0)
    med = np.median(trace, axis=0)
    li = np.percentile(trace, 2.5, axis=0)
    ui = np.percentile(trace, 97.5, axis=0)
    lq = np.percentile(trace, 25, axis=0)
    uq = np.percentile(trace, 75, axis=0)
    return means, med, li, ui, lq, uq

def add_trace_to_plot(res, y_off, col, label, alpha, width, size=8, zero_inds=None):
    nS, _ = res.shape
    nF = len(derived_features)
    derived_samples = np.zeros((nS, nF))

    for f_i, (f, prodrows) in enumerate(derived_features):
        samples = np.ones(nS)
        for r in prodrows:
            samples = samples * res[:, r] 
        derived_samples[:, f_i] = samples

    res = derived_samples
    res = 100*(1-res)
    
    if zero_inds is not None:
        for z in zero_inds:
            res[:, z] = 1e5
            
    y_vals = -1 * np.arange(nF)
    plt.plot([100], [100], color=col, linewidth=1, alpha=alpha, label=label)
    mn, med, li, ui, lq, uq = produce_ranges(res)
    plt.scatter(med, y_vals+y_off, marker="|", color=col, s=size, alpha=alpha)
    for cm in range(nF):
        if cm > 7:
            plt.plot([li[cm], ui[cm]], [y_vals[cm]+y_off, y_vals[cm]+y_off], color=col, alpha=alpha*0.25*0.25, linewidth=width, linestyle="--")
            plt.plot([lq[cm], uq[cm]], [y_vals[cm]+y_off, y_vals[cm]+y_off], color=col, alpha=alpha*0.75*0.25, linewidth=width, linestyle="--")
        else:
            plt.plot([li[cm], ui[cm]], [y_vals[cm]+y_off, y_vals[cm]+y_off], color=col, alpha=alpha*0.25, linewidth=width)
            plt.plot([lq[cm], uq[cm]], [y_vals[cm]+y_off, y_vals[cm]+y_off], color=col, alpha=alpha*0.75, linewidth=width)
        
def setup_large_plot(y_ticks = True, icons = True, xlabel=True, line=True):
    nF = len(derived_features)
    ax = plt.gca()
    x_min = -50
    x_max = 100
    plt.plot([0, 0], [1, -(nF)], "--k", linewidth=0.5)
    xrange = np.array([x_min, x_max])
    for height in range(0, nF+2, 2):
        plt.fill_between(xrange, -(height-0.5), -(height+0.5), color="silver", alpha=0.5, linewidth=0)
    xtick_vals = [-50, -25, 0, 25, 50, 75, 100]
    xtick_str = [f"{x:.0f}%" for x in xtick_vals]
    if y_ticks:
        plt.yticks(-np.arange(nF), [f"{f[0]}" for f in derived_features], fontsize=8, ha="left")
        yax = ax.get_yaxis()
        yax.set_tick_params(pad=140)
        
    else:
        plt.yticks([])
    x_r = np.abs(x_min - x_max)
    plt.xticks(xtick_vals, xtick_str, fontsize=8)
    plt.xlim([x_min, x_max])
    plt.ylim([-(nF - 0.25), 0.75])
    
    if icons:
        for cm in range(len(derived_features)):
            for i, val in enumerate(derived_features[cm][1]):
                plt.text(x_min - 7.5 - 8*i, -cm, cm_plot_style[val][0], horizontalalignment='center', verticalalignment='center',
                             fontproperties=fp2, fontsize=8, color=cm_plot_style[val][1], zorder=-i, alpha = 1) 
                
    if xlabel:
        plt.xlabel("Average additional reduction in $R$, in the context of our data", fontsize=8)
    
    if line:
        plt.plot([x_min, x_max], [-7.5, -7.5], "k", linewidth=2)
        
class ResultsObject():
    def __init__(self, indx, trace):
        self.CMReduction = trace.CMReduction
        self.RegionR = trace.RegionR[:, indx]
        self.InfectedCases = trace.InfectedCases[:, indx, :]
        self.InfectedDeaths = trace.InfectedDeaths[:, indx, :]
        self.ExpectedCases = trace.ExpectedCases[:, indx, :]
        self.ExpectedDeaths = trace.ExpectedDeaths[:, indx, :]

In [7]:
from collections import defaultdict

nCMs = 9

def grab_traces_categorised(fname, verbose=False):
    global default_res
    
    files_found = 0
    all_traces = []
    cat_sen = defaultdict(list)
    for i, (dirpath, dirnames, filenames) in enumerate(walk(fname)):
        for f in filenames:
            if "_combined" in f and ".txt" in f and "rhat" not in f and "ess" not in f and "base" not in f:
                    
                trace = np.loadtxt(dirpath+"/"+f)
                nS, f_nCMs = trace.shape
                
                if f_nCMs > nCMs:
                    trace = trace[:, :nCMs]
                elif f_nCMs < nCMs:
                    leaveout_num = int(f[-5])
                    full_trace = np.zeros((nS, nCMs))
                    full_trace[:, :leaveout_num] = trace[:, :leaveout_num]
#                     set the missing ones to no effect
                    full_trace[:, leaveout_num] = np.ones(nS)
                    full_trace[:, (leaveout_num+1):] = trace[:, leaveout_num:]
                    trace = full_trace
                
                files_found += 1
                
                res = trace
                    
                if "additive" in fname:
                    res = 1 - res
                    
                if "delay_mean_death" in f: 
                    cat_sen["Death Delay"].append((f, res))
                    files_found += 1
                
                    if verbose:
                        print(f"delays added {f}")
                
                if  "delay_mean_confirmed" in f: 
                    cat_sen["Cases Delay"].append((f, res))
                    files_found += 1
                
                    if verbose:
                        print(f"delays added {f}")
                    
                elif "serial_int" in f: 
                    cat_sen["Serial Interval"].append((f, res))
                    files_found += 1
                    
                    if verbose:
                        print(f"si added {f}")
                    
                elif "leavout" in f: 
                    cat_sen["NPI Leaveouts"].append((f, res))
                    files_found += 1
                    
                    if verbose:
                        print(f"leavouts added {f}")
                
                elif "R_hyperprior" in f: 
                    cat_sen["R Priors"].append((f, res))
                    files_found += 1
                    
                    if verbose:
                        print(f"epi prior {f}")
                        
                elif "cm_prior" in f: 
                    cat_sen["NPI Priors"].append((f, res))
                    files_found += 1
                    
                    if verbose:
                        print(f"epi prior {f}")
                        
                elif "noise" in f and "no_noise" not in f: 
                    cat_sen["Hyperparameter"].append((f, res))
                    files_found += 1
                    
                    if verbose:
                        print(f"hypers added {f}")
                        
                elif "regions_heldout" in f or "regholdout" in f: 
                    cat_sen["Region Holdouts"].append((f, res))
                    files_found += 1
                    
                    if verbose:
                        print(f"reg holdouts added {f}")
                    
                elif "schools" in f: 
                    cat_sen["SE Schools"].append((f, res))
                    files_found += 1
                    
                    if verbose:
                        print(f"mob added {f}")
                    
                elif "min_confirmed" in f: 
                    cat_sen["Minimum Confirmed Cases"].append((f, res))
                    files_found += 1
                    
                    if verbose:
                        print(f"min conf added {f}")
                        
                elif "min_deaths" in f: 
                    cat_sen["Minimum Deaths"].append((f, res))
                    files_found += 1
                    
                    if verbose:
                        print(f"min deaths added {f}")
                        
                elif "smoothing" in f: 
                    cat_sen["Smoothing"].append((f, res))
                    files_found += 1
                    
                    if verbose:
                        print(f"smooting {f}")
                
                elif "mobility" in f: 
                    cat_sen["Mobility"].append((f, res))
                    files_found += 1
                    
                    if verbose:
                        print(f"min conf added {f}")
                else:
                    print(f"File {f} not added - no category found")
        
    return cat_sen

def fname_to_label(f):
    if "regholdout" in f:
        return f[:2]
    
    if "regions_heldout" in f:
        return f[16:18]
    
    if "cm_prior" in f:
        if "wide" in f:
            return "Wide"
        
        if "half_normal" in f:
            return "Half Normal"
        
        if "icl" in f:
            return "Flaxman et. al."
    
    if "data_mobility" in f:
        if "no_work" in f:
            return "Retail & Recreation"
        elif "rec_work" in f:
            return "Workplace & Retail &\nRecreation"
    
    if "R_hyperprior" in f:
        if "0" in f:
            return "$\mu=2.5$"
        elif "1" in f:
            return "$\mu=4.5$"
    
    if "schools_open" in f:
        return "SE Schools Open"

    if "delay" in f and "confirmed" in f:
         if "0" in f:
            return "Cases Delay: $-4$ days"
         if "1" in f:
            return "Cases Delay: $-2$ days" 
         if "2" in f:
            return "Cases Delay: $+2$ days" 
         if "3" in f and "v3" not in f:
            return "Cases Delay: $+4$ days" 
    
    if "delay" in f and "death" in f:
         if "0" in f:
            return "Deaths Delay: $-4$ days"
         if "1" in f:
            return "Deaths Delay: $-2$ days" 
         if "2" in f:
            return "Deaths Delay: $+2$ days" 
         if "3" in f and "v3" not in f:
            return "Deaths Delay: $+4$ days" 

    if "cm_leavout" in f:
        if "0" in f:
            return "Mask-wearing"
        if "1" in f:
            return "Gatherings <1000"
        if "2" in f and "v3" not in f:
            return "Gatherings <100"
        if "3" in f:
            return "Gatherings <10"
        if "4" in f:
            return "Some businesses suspended"
        if "5" in f:
            return "Most businesses suspended"
        if "6" in f:
            return "Schools closed"
        if "7" in f:
            return "Universities closed"
        if "8" in f:
            return "Stay-at-home order"
        
    if "serial_int" in f:
        if "SI4" in f:
            return "$\mu=4$"
        if "SI5" in f:
            return "$\mu=5$"
        if "SI6" in f:
            return "$\mu=6$"
        if "SI7" in f:
            return "$\mu=7$"
        if "SI8" in f:
            return "$\mu=8$"
    
    if "min_confirmed" in f:
        if "v3" in f:
            val = re.findall('\\d+', f)[1]
        else:
            val = re.findall('\\d+', f)[0]
        return f"{val}"
    
    if "min_deaths" in f:
        if "v3" in f:
            val = re.findall('\\d+', f)[1]
        else:
            val = re.findall('\\d+', f)[0]
        return f"{val}"
    
    if "cm_prior_combined_additive" in f:
        if "10" in f:
            return "$\\alpha_i\sim$Dirichlet($\\alpha=10$)"
        if "5" in f:
            return "$\\alpha_i\sim$Dirichlet($\\alpha=5$)"
    
    if "smoothing" in f:
        if "v3" in f:
            val = re.findall('\\d+', f)[1]
        else:
            val = re.findall('\\d+', f)[0]
        return f"{val} days"
    
    if "growth_noise" in f and "0" in f:
        return "$\sigma_n=0.05$"
    
    if "growth_noise" in f and "1" in f:
        return "$\sigma_n=0.1$"
    
    if "growth_noise" in f and "2" in f:
        return "$\sigma_n=0.4$"
    
    return f

In [8]:
cat_sen = grab_traces_categorised("../../server/traces")

File delay_mean_death_combined_2.txt not added - no category found
File delay_mean_death_combined_3.txt not added - no category found
File delay_mean_death_combined_1.txt not added - no category found
File delay_mean_death_combined_0.txt not added - no category found


# Category 1: Structural Sensitivity

In [9]:
exp_add = 1
exp_noisyr = 2
exp_diff = 3
exp_dr = 4

res_add = load_exp(exp_add)
res_noisyr = load_exp(exp_noisyr, local=True)
res_diff = load_exp(exp_diff)
res_dr = load_exp(exp_dr)

In [ ]:
fig = plt.figure(figsize=(4, 3), dpi=300)

setup_large_plot()

y_off = -np.linspace(-0.3, 0.3, 5)
width = 1
add_trace_to_plot(1-res_add.CMReduction, y_off[0], colors[0], "Additive", 1, width) 
add_trace_to_plot(res_noisyr.CMReduction, y_off[1], colors[1], "Noisy-R", 1, width) 
add_trace_to_plot(res_diff.CMReduction, y_off[2], colors[2], "Different Effects", 1, width)
add_trace_to_plot(res_dr.CMReduction, y_off[3], colors[3], "Discrete Renewal", 1, width)
add_trace_to_plot(default_res, y_off[4], "k", "Default", 1, width) 
plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.2, 1.01), fontsize=6)
plt.title("Structural Sensitivity", fontsize="10")

plt.savefig(f"FigureSSA.pdf", bbox_inches='tight')

# Category 2: Unobserved Effects

In [ ]:
exp_travel = 5
exp_transport = 6
exp_int_movement = 7
exp_info = 8
exp_testing = 9
exp_schools_uni = 20


res_travel = load_exp(exp_travel)
res_transport = load_exp(exp_transport)
res_int_movement = load_exp(exp_int_movement)
res_info = load_exp(exp_info)
res_testing = load_exp(exp_testing)

res_su = load_exp(exp_schools_uni, True)

In [ ]:
fig = plt.figure(figsize=(10, 3), dpi=300)

plt.subplot(121)
setup_large_plot(xlabel=False)

y_off = -np.linspace(-0.3, 0.3, 6)
width = 1
add_trace_to_plot(res_travel.CMReduction[:, :9], y_off[0], colors[0], "Travel Screening/Bans", 1, width) 
add_trace_to_plot(res_transport.CMReduction[:, :9], y_off[1], colors[1], "Public Transport Limited", 1, width) 
add_trace_to_plot(res_int_movement.CMReduction[:, :9], y_off[2], colors[2], "Internal Movement Limited", 1, width)
add_trace_to_plot(res_info.CMReduction[:, :9], y_off[3], colors[3], "Public Information Campaigns", 1, width)
add_trace_to_plot(res_testing.CMReduction[:, [0, 2, 3, 4, 5, 6, 7, 8, 9]], y_off[4], colors[4], "Symptomatic Testing", 1, width)
add_trace_to_plot(default_res, y_off[5], "k", "Default", 1, width) 

plt.title("Inclusion of OxCGRT NPIs", fontsize=10)
plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.2, 1.01), fontsize=6)

# plt.subplot(132)
# setup_large_plot(False)

# y_off = -np.linspace(-0.3, 0.3, 4)
# width = 1
# add_trace_to_plot(res_mob_work.CMReduction[:, :9], y_off[0], colors[0], "Workplace Mobility", 1, width) 
# add_trace_to_plot(res_mob_retail.CMReduction[:, :9], y_off[1], colors[1], "Retail and Recreation\n Mobility", 1, width) 
# add_trace_to_plot(res_mob_both.CMReduction[:, :9], y_off[2], colors[2], "Workplace, Retail and\nRecreation Mobility", 1, width)
# add_trace_to_plot(default_res, y_off[3], "k", "Default", 1, width) 

# plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.2, 1.01), fontsize=6)

plt.subplot(122)
setup_large_plot(False)

cat_sen["NPI Leaveouts"].sort(key = lambda x: x[0])

y_off = -np.linspace(-0.4, 0.4, len(cat_sen["NPI Leaveouts"])+1)

for i, (f, t) in enumerate(cat_sen["NPI Leaveouts"]):
    if i == 6:
        zds = [6, 8]
    elif i == 7 :
        zds = [6, 9]
    else:
        zds = [i]
    add_trace_to_plot(t, y_off[i], colors[i], fname_to_label(f), 1, width, zero_inds = zds)

cm_red = res_su.CMReduction
nS, _ = cm_red.shape
cm_red[:, 6] = np.ones(nS)
cm_red[:, 7] = np.ones(nS)
add_trace_to_plot(cm_red, y_off[-1], colors[-1], "School and University Closure", 1, width, zero_inds = [6, 8, 9])
plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.4, 1.01), fontsize=6)

plt.title("Exclusion of Collected NPIs", fontsize=10)
plt.tight_layout()
plt.savefig(f"FigureUOB.pdf", bbox_inches='tight')

In [ ]:
derived_features = [ #()"Healthcare Infection Control", [0]),
 ("Mask-wearing", [0]),
 ("Gatherings <1000", [1]),
 ("Gatherings <100", [2, 1]),
 ("Gatherings <10", [3, 2, 1]),
 ("Some businesses closed", [4]),
 ("Most businesses closed", [4, 5]),
 ("Schools and universities\nclosed", [6, 7]),
 ("Stay-at-home order", [8]),
 ("Schools closed", [6]),
 ("Universities closed", [7]),
 ]


fig = plt.figure(figsize=(10, 3), dpi=300)

plt.subplot(121)
setup_large_plot(xlabel=False)

y_off = -np.linspace(-0.3, 0.3, 6)
width = 1
add_trace_to_plot(res_travel.CMReduction[:, :9], y_off[0], colors[0], "Travel Screening/Bans", 1, width) 
add_trace_to_plot(res_transport.CMReduction[:, :9], y_off[1], colors[1], "Public Transport Limited", 1, width) 
add_trace_to_plot(res_int_movement.CMReduction[:, :9], y_off[2], colors[2], "Internal Movement Limited", 1, width)
add_trace_to_plot(res_info.CMReduction[:, :9], y_off[3], colors[3], "Public Information Campaigns", 1, width)
add_trace_to_plot(res_testing.CMReduction[:, [0, 2, 3, 4, 5, 6, 7, 8, 9]], y_off[4], colors[4], "Symptomatic Testing", 1, width)
add_trace_to_plot(default_res, y_off[5], "k", "Default", 1, width) 

plt.title("Inclusion of OxCGRT NPIs", fontsize=10)
plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.2, 1.01), fontsize=6)

# plt.subplot(132)
# setup_large_plot(False)

# y_off = -np.linspace(-0.3, 0.3, 4)
# width = 1
# add_trace_to_plot(res_mob_work.CMReduction[:, :9], y_off[0], colors[0], "Workplace Mobility", 1, width) 
# add_trace_to_plot(res_mob_retail.CMReduction[:, :9], y_off[1], colors[1], "Retail and Recreation\n Mobility", 1, width) 
# add_trace_to_plot(res_mob_both.CMReduction[:, :9], y_off[2], colors[2], "Workplace, Retail and\nRecreation Mobility", 1, width)
# add_trace_to_plot(default_res, y_off[3], "k", "Default", 1, width) 

# plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.2, 1.01), fontsize=6)

plt.subplot(122)
derived_features = [ #()"Healthcare Infection Control", [0]),
 ("Mask-wearing", [0]),
 ("Gatherings <1000", [1]),
 ("Gatherings <100", [2]),
 ("Gatherings <10", [3]),
 ("Some businesses closed", [4]),
 ("Most businesses closed", [5]),
 ("Schools closed", [6]),
 ("Universities closed", [7]),
 ("Stay-at-home order", [8]),
 ]
setup_large_plot(False)

cat_sen["NPI Leaveouts"].sort(key = lambda x: x[0])

y_off = -np.linspace(-0.4, 0.4, len(cat_sen["NPI Leaveouts"])+1)

for i, (f, t) in enumerate(cat_sen["NPI Leaveouts"]):
    if i == 6:
        zds = [6, 8]
    elif i == 7 :
        zds = [6, 9]
    else:
        zds = [i]
    print(i)
    print(fname_to_label(f))
    zds = [i]
    add_trace_to_plot(t, y_off[i], colors[i], fname_to_label(f), 1, width, zero_inds = zds)

cm_red = res_su.CMReduction
nS, _ = cm_red.shape
cm_red[:, 6] = np.ones(nS)
cm_red[:, 7] = np.ones(nS)
add_trace_to_plot(cm_red, y_off[-1], colors[-1], "School and University Closure", 1, width, zero_inds = [6, 7])
plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.4, 1.01), fontsize=6)
plt.ylim([-7.5, 0.5])

plt.title("Exclusion of Collected NPIs\nNote: additional (not cumulative) effectiveness shown", fontsize=10)
plt.tight_layout()
plt.savefig(f"FigureUOB_marginal.pdf", bbox_inches='tight')

derived_features = [ #()"Healthcare Infection Control", [0]),
 ("Mask-wearing", [0]),
 ("Gatherings <1000", [1]),
 ("Gatherings <100", [2, 1]),
 ("Gatherings <10", [3, 2, 1]),
 ("Some businesses closed", [4]),
 ("Most businesses closed", [4, 5]),
 ("Schools and universities\nclosed", [6, 7]),
 ("Stay-at-home order", [8]),
 ("Schools closed", [6]),
 ("Universities closed", [7]),
 ]

# Category 3: Data

In [ ]:
exp_active = 18
exp_death = 19

res_active = load_exp(exp_active)
res_death = load_exp(exp_death)

In [ ]:
fig = plt.figure(figsize=(10, 6), dpi=300)

plt.subplot(221)
setup_large_plot()
y_off = -np.linspace(-0.2, 0.2, 3)
width = 2
add_trace_to_plot(res_active.CMReduction, y_off[0], colors[0], "Confirmed Cases", 1, width) 
add_trace_to_plot(res_death.CMReduction, y_off[1], colors[1], "Confirmed Deaths", 1, width) 
add_trace_to_plot(default_res, y_off[2], "k", "Default", 1, width)
plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.2, 1.01), fontsize=6)
plt.title("Data Source", fontsize=10)

width=1
plt.subplot(222)
setup_large_plot(False)
res = cat_sen["Smoothing"]
res.sort(key = lambda x: int(fname_to_label(x[0])[:2]))

y_off = -np.linspace(-0.3, 0.3, len(res)+1)

add_trace_to_plot(res[0][1], y_off[0], colors[0], fname_to_label(res[0][0]), 1, width)
add_trace_to_plot(res[1][1], y_off[1], colors[1], fname_to_label(res[1][0]), 1, width)
add_trace_to_plot(default_res, y_off[2], "k", "Default (5)", 1, width)
add_trace_to_plot(res[2][1], y_off[3], colors[2], fname_to_label(res[2][0]), 1, width)
add_trace_to_plot(res[3][1], y_off[4], colors[3], fname_to_label(res[3][0]), 1, width)
plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.2, 1.01), fontsize=6)
plt.title("Smoothing", fontsize=10)

plt.subplot(223)
setup_large_plot()
res = cat_sen["Minimum Deaths"]
res.sort(key = lambda x: int(fname_to_label(x[0])[:4]))

y_off = -np.linspace(-0.3, 0.3, len(res)+1)

add_trace_to_plot(res[0][1], y_off[0], colors[0], fname_to_label(res[0][0]), 1, width)
add_trace_to_plot(res[1][1], y_off[1], colors[1], fname_to_label(res[1][0]), 1, width)
add_trace_to_plot(default_res, y_off[2], "k", "Default (10)", 1, width)
add_trace_to_plot(res[2][1], y_off[3], colors[2], fname_to_label(res[2][0]), 1, width)
add_trace_to_plot(res[3][1], y_off[4], colors[3], fname_to_label(res[3][0]), 1, width)
plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.2, 1.01), fontsize=6)

plt.title("Minimum Deaths Threshold", fontsize=10)
plt.subplot(224)
setup_large_plot(False)

res = cat_sen["Minimum Confirmed Cases"]
res.sort(key = lambda x: int(fname_to_label(x[0])[:4]))

y_off = -np.linspace(-0.3, 0.3, len(res)+1)

add_trace_to_plot(res[0][1], y_off[0], colors[0], fname_to_label(res[0][0]), 1, width)
add_trace_to_plot(res[1][1], y_off[1], colors[1], fname_to_label(res[1][0]), 1, width)
add_trace_to_plot(default_res, y_off[2], "k", "Default (100)", 1, width)
add_trace_to_plot(res[2][1], y_off[3], colors[2], fname_to_label(res[2][0]), 1, width)
add_trace_to_plot(res[3][1], y_off[4], colors[3], fname_to_label(res[3][0]), 1, width)
plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.2, 1.01), fontsize=6)
plt.title("Minimum Confirmed Cases Threshold", fontsize=10)

plt.tight_layout()
plt.savefig(f"FigureD1.pdf", bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(9, 9), dpi=300)

y_off = -np.linspace(-0.4, 0.4, 8)
for r_i, r in enumerate(data.Rs):
    r_i_p = r_i % 7 
    if r_i_p == 0:
        plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.2, 1.01), fontsize=6)
        plt.subplot(3, 2, int(r_i/7) + 1)
        if (int(r_i/7) + 1) == 1 or (int(r_i/7) + 1) == 3 or (int(r_i/7) + 1) == 5:
            setup_large_plot()
        else:
            setup_large_plot(False)
        add_trace_to_plot(default_res, y_off[-1], "k", "Default", 1, 1)
    res = pickle.load(open(f"../../server/ho_results_final4/{r}.pkl", "rb")).CMReduction
    add_trace_to_plot(res, y_off[r_i_p], colors[r_i_p], data.RNames[r][0], 1, 1)
plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.2, 1.01), fontsize=6)
plt.tight_layout()
plt.savefig(f"FigureD2.pdf", bbox_inches='tight')

# Category 4: NPI timing

In [ ]:
exp_any_active = 13
exp_mm = 14
exp_difdelay = 15
exp_schools_delay = 16

res_any_active = load_exp(exp_any_active)
res_mm = load_exp(exp_mm, True)
res_difdelay = load_exp(exp_difdelay)
res_schoolsdelay = load_exp(exp_schools_delay)

In [ ]:
fig = plt.figure(figsize=(10, 3.5), dpi=300)

plt.subplot(121)
setup_large_plot()

y_off = -np.linspace(-0.3, 0.3, 4)
width = 1
add_trace_to_plot(res_any_active.CMReduction, y_off[0], colors[0], "Any Active\nBonus NPI", 1, width) 
add_trace_to_plot(res_difdelay.CMReduction[:, [0, 2, 3, 4, 5, 6, 7, 8, 9]], y_off[1], colors[1], "Different Confirmation\nDelays", 1, width) 
add_trace_to_plot(res_schoolsdelay.CMReduction, y_off[2], colors[2], "Schools and\nUniversities Delayed", 1, width)
add_trace_to_plot(default_res, y_off[3], "k", "Default", 1, width) 
plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.2, 1.01), fontsize=6)
plt.title("NPI Timing 1", fontsize=10)

plt.subplot(122)
setup_large_plot(False, False, line=False)
width = 2
npis = [f"NPI {i+1}" for i in range(9)]
plt.yticks(-np.arange(len(npis)), npis, fontsize=8)
plt.ylim([-(len(npis) - 0.25), 0.75])

y_vals = -1 * np.arange(len(npis))
plt.plot([100], [100], color=colors[0], linewidth=1, alpha=1, label="")
mn, med, li, ui, lq, uq = produce_ranges(100*(1-res_mm.CMReduction))
plt.scatter(med, y_vals, marker="|", color=colors[0], s=16, alpha=1)
for cm in range(len(npis)):
    plt.plot([li[cm], ui[cm]], [y_vals[cm], y_vals[cm]], color=colors[0], alpha=0.25, linewidth=width)
    plt.plot([lq[cm], uq[cm]], [y_vals[cm], y_vals[cm]], color=colors[0], alpha=0.75, linewidth=width)

plt.tight_layout()
plt.title("NPI Timing 2", fontsize=10)
plt.savefig(f"FigureNPITiming.pdf", bbox_inches='tight')

# Category 5: epidemiological

In [ ]:
fig = plt.figure(figsize=(9, 9), dpi=300)

width=1
plt.subplot(321)
cat = "Serial Interval"
setup_large_plot()
cat_sen[cat].sort(key = lambda x: f)

y_off = -np.linspace(-0.3, 0.3, len(cat_sen[cat])+1)

for i, (f, t) in enumerate(cat_sen[cat]):
    add_trace_to_plot(t, y_off[i], colors[i], fname_to_label(f), 1, width)
add_trace_to_plot(default_res, y_off[-1], "k", "Default ($\mu = 6.67$)", 1, width)
plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.2, 1.01), fontsize=6)
plt.title("Generation Interval", fontsize=10)

plt.subplot(322)
cat = "NPI Priors"
setup_large_plot(False)
cat_sen[cat].sort(key = lambda x: f)

y_off = -np.linspace(-0.3, 0.3, len(cat_sen[cat])+1)

for i, (f, t) in enumerate(cat_sen[cat]):
    add_trace_to_plot(t, y_off[i], colors[i], fname_to_label(f), 1, width)
add_trace_to_plot(default_res, y_off[-1], "k", "Default", 1, width)
plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.2, 1.01), fontsize=6)

plt.title("Effectiveness Prior", fontsize=10)
plt.subplot(323)
cat = "R Priors"
setup_large_plot()
cat_sen[cat].sort(key = lambda x: f)

y_off = -np.linspace(-0.3, 0.3, len(cat_sen[cat])+1)

for i, (f, t) in enumerate(cat_sen[cat]):
    add_trace_to_plot(t, y_off[i], colors[i], fname_to_label(f), 1, width)
add_trace_to_plot(default_res, y_off[-1], "k", "Default ($\mu=3.25$)", 1, width)
plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.2, 1.01), fontsize=6)
plt.title("$R_0$ prior", fontsize=10)

plt.subplot(324)
cat = "Cases Delay"
setup_large_plot(False)
cat_sen[cat].sort(key = lambda x: f)

y_off = -np.linspace(-0.3, 0.3, len(cat_sen[cat])+1)

for i, (f, t) in enumerate(cat_sen[cat]):
    add_trace_to_plot(t, y_off[i], colors[i], fname_to_label(f), 1, width)
add_trace_to_plot(default_res, y_off[-1], "k", "Default", 1, width)
plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.2, 1.01), fontsize=6)
plt.title("Reporting Delay", fontsize=10)

plt.subplot(325)
cat = "Death Delay"
setup_large_plot()
cat_sen[cat].sort(key = lambda x: f)

y_off = -np.linspace(-0.3, 0.3, len(cat_sen[cat])+1)

for i, (f, t) in enumerate(cat_sen[cat]):
    add_trace_to_plot(t, y_off[i], colors[i], fname_to_label(f), 1, width)
add_trace_to_plot(default_res, y_off[-1], "k", "Default", 1, width)
plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.2, 1.01), fontsize=6)
plt.title("Death Delay", fontsize=10)

plt.subplot(326)
cat = "Hyperparameter"
setup_large_plot(False)
cat_sen[cat].sort(key = lambda x: f)

y_off = -np.linspace(-0.3, 0.3, len(cat_sen[cat])+1)

for i, (f, t) in enumerate(cat_sen[cat]):
    add_trace_to_plot(t, y_off[i], colors[i], fname_to_label(f), 1, width)
add_trace_to_plot(default_res, y_off[-1], "k", "$\sigma_n = 0.2$", 1, width)
plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.2, 1.01), fontsize=6)
plt.title("Growth Noise", fontsize=10)

plt.tight_layout()
plt.savefig(f"FigureEpi.pdf", bbox_inches='tight')

# Schools/Unis Figure

In [ ]:
plt.figure(figsize=(4,3), dpi=300)

derived_features = [ #()"Healthcare Infection Control", [0]),
 ("Schools closed", [6]),
 ("Universities closed", [7]),
 ("Schools and univerisities closed", [6, 7]),
 ]

nF = len(derived_features)

ax = plt.gca()
x_min = -50
x_max = 100
plt.plot([0, 0], [1, -(nF)], "--k", linewidth=0.5)
xrange = np.array([x_min, x_max])
for height in range(0, nF+2, 2):
    plt.fill_between(xrange, -(height-0.5), -(height+0.5), color="silver", alpha=0.5, linewidth=0)
xtick_vals = [-50, -25, 0, 25, 50, 75, 100]
xtick_str = [f"{x:.0f}%" for x in xtick_vals]

plt.yticks(-np.arange(nF), [f"{f[0]}" for f in derived_features], fontsize=8, ha="left")
yax = ax.get_yaxis()
yax.set_tick_params(pad=140)

x_r = np.abs(x_min - x_max)
plt.xticks(xtick_vals, xtick_str, fontsize=8)
plt.xlim([x_min, x_max])
plt.ylim([-(nF - 0.25), 0.75])


for cm in range(len(derived_features)):
    for i, val in enumerate(derived_features[cm][1]):
        plt.text(x_min - 7.5 - 8*i, -cm, cm_plot_style[val][0], horizontalalignment='center', verticalalignment='center',
                     fontproperties=fp2, fontsize=8, color=cm_plot_style[val][1], zorder=-i, alpha = 1) 

plt.xlabel("Average additional reduction in $R$, in the context of our data", fontsize=8)


iceland = pickle.load(open(f"../../server/ho_results_final4/IS.pkl", "rb")).CMReduction
sweden = pickle.load(open(f"../../server/ho_results_final4/SE.pkl", "rb")).CMReduction
y_off = -np.linspace(-0.2, 0.2, 3)
add_trace_to_plot(iceland, y_off[0], colors[0], "Iceland Excluded", 1, 1)
add_trace_to_plot(sweden, y_off[1], colors[1], "Sweden Excluded", 1, 1)
add_trace_to_plot(default_res, y_off[-1], "k", "Default", 1, 1)
plt.legend(shadow=True, fancybox=True, fontsize=6)
plt.title("Schools and Universities Sensitivity", fontsize=10)
plt.savefig(f"FigureSwedIce.pdf", bbox_inches='tight')

In [ ]:
data.RNames["SE"]

In [ ]:
data.RNames["IS"]